# Признаки: категориальные и числовые

> Под __числовыми признаками__ обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.  

Числовые признаки могут быть:
- дискретными (например, количество комнат, пациентов, дней, отток сотрудников);
- непрерывными (например, масса, цена, площадь).
>>Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.
___

> Под __категориальными признаками__ обычно подразумевают столбцы в таблице, которые обозначают принадлежность объекта к какому-то классу/категории.  

Категориальные признаки могут быть:
- номинальными (например, пол, национальность, район);
- порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).
>> Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.
>>>Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.
___

>__Анализ и предобработка категориальных признаков отличается от предобработки числовых признаков.__

## Категории в данных о недвижимости

In [99]:
import pandas as pd

In [100]:
melb_df = pd.read_csv('data/melb_data_dt.csv', sep=',')

In [101]:
unique_list = []
for i_col in melb_df.columns:
    item = (i_col, melb_df[i_col].nunique(), melb_df[i_col].dtypes)
    unique_list.append(item)
unique_df = pd.DataFrame(unique_list,
                         columns=['Column_Name', 'Num_Unique', 'Type']
                         ).sort_values('Num_Unique', ignore_index=True)

display(unique_df)

,Column_Name,Num_Unique,Type
0,Weekday,2,int64
1,YearSold,2,int64
2,Type,3,object
3,WeekdaySale,5,int64
4,Method,5,object
5,Regionname,8,object
6,Rooms,9,int64
7,Bathroom,9,int64
8,Car,11,int64
9,Bedroom,12,int64


- Если присмотреться внимательно, можно увидеть резкий скачок количества уникальных значений, начиная с 14 строки таблицы, где число уникальных значений составляет 152. __Учтём этот момент.__

> Условимся, что __категориальными__ будем считать признаки, у которых __число уникальных категорий меньше 150.__ 

Однако учтём, что __признак Date (дата продажи)__, преобразованный нами ранее в формат datetime, является __временным признаком__, поэтому далее __не будем его воспринимать как категориальный.__

> К тому же в наш потенциальный список попали __количественные столбцы Rooms, Car, Bedroom и Bathroom.__ Договоримся, что мы __не будем относить__ их к разряду категориальных, __однако такое тоже вполне возможно.__

>Примечание. Ещё раз подчеркиваем, что такая классификация признаков является исключительно субъективной и специфична для задачи.

# Тип данных category

- Для хранения и оптимизации работы с категориальными признаками в Pandas предусмотрен специальный тип данных — category.

>→ Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. Так как данные вместо изначальных строк хранятся в памяти как число, то объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается, что повышает эффективность хранения и работы с таблицей.

>→ Более того, этот тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным (что сложно сделать для типа данных object). Также резко повышается производительность операций, совершаемых с такими столбцами.

In [102]:
# обратить внимание на объём памяти
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Address        13580 non-null  object 
 3   Rooms          13580 non-null  int64  
 4   Type           13580 non-null  object 
 5   Price          13580 non-null  float64
 6   Method         13580 non-null  object 
 7   SellerG        13580 non-null  object 
 8   Date           13580 non-null  object 
 9   Distance       13580 non-null  float64
 10  Postcode       13580 non-null  int64  
 11  Bedroom        13580 non-null  int64  
 12  Bathroom       13580 non-null  int64  
 13  Car            13580 non-null  int64  
 14  Landsize       13580 non-null  float64
 15  BuildingArea   13580 non-null  float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

None

In [ ]:
# преобразуем тип данных в category
# обратить внимание на объём памяти
exclude_colu = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_uniq_cnt = 150
for i_col in melb_df.columns:
    if melb_df[i_col].nunique() < (max_uniq_cnt) and i_col not in exclude_colu:
        melb_df[i_col] = melb_df[i_col].astype('category')

display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  CouncilArea    12211 non-null  category
 17  Lattitude      13580 non-null  

None

## Получение атрибутов category

- У типа данных __category__ есть свой специальный __аксесcор cat__, который позволяет получать __информацию о своих значениях и преобразовывать их__. Например, с помощью __атрибута__ этого аксессора __categories__ мы можем получить __список уникальных категорий__

In [104]:
melb_df['Regionname'].cat.categories

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')

- А теперь посмотрим, каким образом __столбец кодируется в виде чисел в памяти компьютера__. Для этого можно воспользоваться __атрибутом codes__:

In [105]:
melb_df['Regionname'].cat.codes

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

In [106]:
melb_df['Type'].info

<bound method Series.info of 0        h
1        h
2        h
3        h
4        h
        ..
13575    h
13576    h
13577    h
13578    h
13579    h
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['h', 't', 'u']>

- С помощью __метода__ аксессора __rename_categories()__ можно легко __переименовать текущие значения категорий__. Для этого в данный метод нужно передать __словарь, ключи которого — старые имена категорий, а значения — новые__.

In [107]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u':'unit', 'h':'house', 't':'townhouse'
    })

melb_df['Type']

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

## Подводные камни

- добавление новых категорийных значений в уже существующий

In [108]:
# обрати внимание на dtype
new_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
print(new_types)
new_types = new_types.astype(melb_df['Type'].dtype)
print(new_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: object
0     unit
1    house
2      NaN
3      NaN
4    house
dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']


> тип данных __category__ хранит только категории, которые __были объявлены при его инициализации__. При __встрече с новой__, неизвестной ранее категорией, __этот тип превратит её в пустое значение__, так как __он просто не знает о существовании этой категории__.

>> Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью __метода__ акссесора cat __add_categories()__, в который достаточно просто __передать имя новой категории__:

In [109]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_types = new_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_types = new_types.astype(melb_df['Type'].dtype)
new_types

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

>  Добавление новой категории в столбец __Type не отразится на самом столбце — текущие категории не изменятся__, однако такое преобразование позволит __добавлять в таблицу новые данные о домах с новой категорией — flat__.

>Из данного примера можно сделать вывод, что если набор категорий в столбце жёстко __не зафиксирован и может обновляться в процессе работы (т.е есть риск внесения в столбец новых данных с несуществующей категорией)__, то тип __category не является подходящим типом данных для этого столбца или необходимо постоянно писать проверки при обновлении таблицы.__

# __Рекомендаций по его использованию CATEGORY:__
1. Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.

2. Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.

3. Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.

4. Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.

In [110]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  object  
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  CouncilArea    12211 non-null  category
 17  Lattitude      13580 non-null  

In [111]:
melb_df['Suburb'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13580 entries, 0 to 13579
Series name: Suburb
Non-Null Count  Dtype 
--------------  ----- 
13580 non-null  object
dtypes: object(1)
memory usage: 106.2+ KB


In [112]:
top_119_surb = melb_df['Suburb'].value_counts().nlargest(119).index
top_119_surb

Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', name='Suburb', length=119)

In [113]:
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in top_119_surb else 'other')

In [114]:
melb_df['Suburb'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13580 entries, 0 to 13579
Series name: Suburb
Non-Null Count  Dtype 
--------------  ----- 
13580 non-null  object
dtypes: object(1)
memory usage: 106.2+ KB


In [ ]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df['Suburb'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 13580 entries, 0 to 13579
Series name: Suburb
Non-Null Count  Dtype   
--------------  -----   
13580 non-null  category
dtypes: category(1)
memory usage: 18.4 KB


In [117]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   index          13580 non-null  int64   
 1   Suburb         13580 non-null  category
 2   Address        13580 non-null  object  
 3   Rooms          13580 non-null  int64   
 4   Type           13580 non-null  category
 5   Price          13580 non-null  float64 
 6   Method         13580 non-null  category
 7   SellerG        13580 non-null  category
 8   Date           13580 non-null  object  
 9   Distance       13580 non-null  float64 
 10  Postcode       13580 non-null  int64   
 11  Bedroom        13580 non-null  int64   
 12  Bathroom       13580 non-null  int64   
 13  Car            13580 non-null  int64   
 14  Landsize       13580 non-null  float64 
 15  BuildingArea   13580 non-null  float64 
 16  CouncilArea    12211 non-null  category
 17  Lattitude      13580 non-null  